In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/riiid-test-answer-prediction/lectures.csv
/kaggle/input/riiid-test-answer-prediction/example_sample_submission.csv
/kaggle/input/riiid-test-answer-prediction/example_test.csv
/kaggle/input/riiid-test-answer-prediction/questions.csv
/kaggle/input/riiid-test-answer-prediction/train.csv
/kaggle/input/riiid-test-answer-prediction/riiideducation/__init__.py
/kaggle/input/riiid-test-answer-prediction/riiideducation/competition.cpython-37m-x86_64-linux-gnu.so


In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
import tensorflow as tf
# 数据地址
dir_path = '/kaggle/input/riiid-test-answer-prediction/'
file_train = 'train.csv'
file_questions = 'questions.csv'

In [5]:
# 导入工具包
# 数据大小
nrows =  100 * 10000
# nrows = None
# 读取训练数据
train = pd.read_csv(
                    dir_path + file_train, 
                    nrows=nrows, 
                    usecols=['row_id', 'timestamp', 'user_id', 'content_id', 
                             'content_type_id', 'task_container_id', 'answered_correctly',
                            'prior_question_elapsed_time','prior_question_had_explanation'],
                    dtype={
                            'row_id': 'int64',
                            'timestamp': 'int64',
                            'user_id': 'int32',
                            'content_id': 'int16',
                            'content_type_id': 'int8',
                            'task_container_id': 'int8',
                            'answered_correctly': 'int8',
                            'prior_question_elapsed_time': 'float32',
                            'prior_question_had_explanation': 'str'
                        }
                   )
# 读取问题数据
questions = pd.read_csv(
                        dir_path + file_questions, 
                        nrows=nrows,
                        usecols=['question_id','bundle_id','part'], 
                        dtype={
                           'question_id': 'int16',
                           'bundle_id': 'int16',
                           'part': 'int8',
                       }
                    )

In [6]:


# prior_question_had_explanation 特征处理
train['prior_question_had_explanation'] = train['prior_question_had_explanation'].map({'True':1,'False':0}).fillna(-1).astype(np.int8)
# 只保留问题数据
train = train[train['content_type_id']==0]
# 释放内存
import gc
gc.collect()
# tailf选取部分数据进行建模，减少内存压力
max_num = 100
train = train.groupby(['user_id']).tail(max_num)
# 训练数据和问题数据进行关联
train = pd.merge(
        left=train,
        right=questions,
        how='left',
        left_on='content_id',
        right_on='question_id'
        )
# 训练数据填充0
train = train.fillna(0)

In [7]:

# 类别特征转换函数
class cat_deal:
    def __init__(self):
        self.max_len = 0
        self.dict_map = {}
    
    def fit(self, cat_list):
        index = 1 
        for cat_i in cat_list:
            if cat_i not in self.dict_map:
                self.dict_map[cat_i] = index
                index += 1
        self.max_len = index + 1
        
    def transform(self, cat_list):
        cat_transform_list = []
        for cat_i in cat_list:
            if cat_i in self.dict_map:
                cat_transform_list.append(self.dict_map[cat_i])
            else:
                cat_transform_list.append(0)
        return cat_transform_list
#%% md
# 浮点特征转换函数
#%%
class float_deal:
    def __init__(self):
        self.max = 0
        self.min = 0
        self.max_min = 0 
        
    def fit(self, float_list):
        for float_i in float_list:
            if float_i < self.min:
                self.min = float_i
            if float_i > self.max:
                self.max = float_i
        self.max_min = self.max - self.min
        
    def transform(self, float_list):
        float_transform_list = []
        for float_i in float_list:
            if float_i < self.min:
                float_transform_list.append(0)
            elif float_i > self.max:
                float_transform_list.append(1)
            else:
                float_transform_list.append(float_i/self.max_min)
        return float_transform_list
    
# 处理类别特征
dict_cat_class = {}
for columns in ['user_id','content_id',\
                'task_container_id','prior_question_had_explanation',\
                'bundle_id','part']:
    dict_cat_class[columns] = cat_deal()
    dict_cat_class[columns].fit(train[columns])

    train[columns] = dict_cat_class[columns].transform(train[columns])
    print(columns)
# str(content_id) + str(part) ---> embedding

# 处理浮点特征
dict_float_class = {}
for columns in ['timestamp','prior_question_elapsed_time']:
    dict_float_class[columns] = float_deal()
    dict_float_class[columns].fit(train[columns])
    
    train[columns] = dict_float_class[columns].transform(train[columns])
    print(columns)

user_id
content_id
task_container_id
prior_question_had_explanation
bundle_id
part
timestamp
prior_question_elapsed_time


In [8]:


# 模型使用的函数（keras Lambda函数包装）
def squeeze(embedding):
    embedding = tf.squeeze(embedding,axis=1)
    return embedding
def concat(embedding_list):
    embedding = tf.concat(embedding_list, axis=1)
    return embedding
def multiply(multi_x_y):
    multi_x = multi_x_y[0]
    multi_y = multi_x_y[1]
    multi_x_y = tf.multiply(multi_x, multi_y)
    return multi_x_y

In [9]:
# 浮点数据输入
input_timestamp = tf.keras.Input(shape=(1,))
input_prior_question_elapsed_time = tf.keras.Input(shape=(1,))

# 类别数据输入
input_user = tf.keras.Input(shape=(1,))
input_content = tf.keras.Input(shape=(1,))
input_task_container = tf.keras.Input(shape=(1,))
input_prior_question_had_explanation = tf.keras.Input(shape=(1,))
input_bundle = tf.keras.Input(shape=(1,))
input_part = tf.keras.Input(shape=(1,))

# 所有输入
inputs = [input_timestamp,input_prior_question_elapsed_time,\
         input_user,input_content,\
         input_task_container,input_prior_question_had_explanation,\
         input_bundle,input_part]

In [10]:
# Wide部分
# 类别特征embeeding转换 
embedding_user_wide = tf.keras.layers.Embedding(dict_cat_class['user_id'].max_len,
                                           1, input_length=1)(input_user)
embedding_user_wide = tf.keras.layers.Lambda(squeeze)(embedding_user_wide)

embedding_content_wide = tf.keras.layers.Embedding(dict_cat_class['content_id'].max_len,
                                              1, input_length=1)(input_content)
embedding_content_wide = tf.keras.layers.Lambda(squeeze)(embedding_content_wide)

embedding_task_container_wide = tf.keras.layers.Embedding(dict_cat_class['task_container_id'].max_len,
                                                     1, input_length=1)(input_task_container)
embedding_task_container_wide = tf.keras.layers.Lambda(squeeze)(embedding_task_container_wide)

embedding_prior_question_had_explanation_wide = tf.keras.layers.Embedding(dict_cat_class['prior_question_had_explanation'].max_len, 
                                                                     1, input_length=1)(input_prior_question_had_explanation)
embedding_prior_question_had_explanation_wide = tf.keras.layers.Lambda(squeeze)(embedding_prior_question_had_explanation_wide)

embedding_bundle_wide = tf.keras.layers.Embedding(dict_cat_class['bundle_id'].max_len,
                                             1, input_length=1)(input_bundle)
embedding_bundle_wide = tf.keras.layers.Lambda(squeeze)(embedding_bundle_wide)

embedding_part_wide = tf.keras.layers.Embedding(dict_cat_class['part'].max_len,
                                           1, input_length=1)(input_part)
embedding_part_wide = tf.keras.layers.Lambda(squeeze)(embedding_part_wide)

# 合并类别特征对应的embeeding特征和浮点特征
embedding_all = [input_timestamp,input_prior_question_elapsed_time,\
                embedding_user_wide, embedding_content_wide, embedding_task_container_wide,\
                embedding_prior_question_had_explanation_wide, embedding_bundle_wide, embedding_part_wide]

wide_all = embedding_all + [input_timestamp,input_prior_question_elapsed_time]

# wide layer
wide_layer = tf.keras.layers.Lambda(concat)(wide_all) 

In [15]:
# Deep部分
# 类别特征embeeding转换 
embedding_user_deep = tf.keras.layers.Embedding(dict_cat_class['user_id'].max_len,
                                           8, input_length=1)(input_user)
embedding_user_deep = tf.keras.layers.Lambda(squeeze)(embedding_user_deep)

embedding_content_deep = tf.keras.layers.Embedding(dict_cat_class['content_id'].max_len,
                                              8, input_length=1)(input_content)
embedding_content_deep = tf.keras.layers.Lambda(squeeze)(embedding_content_deep)

embedding_task_container_deep = tf.keras.layers.Embedding(dict_cat_class['task_container_id'].max_len,
                                                     8, input_length=1)(input_task_container)
embedding_task_container_deep = tf.keras.layers.Lambda(squeeze)(embedding_task_container_deep)

embedding_prior_question_had_explanation_deep = tf.keras.layers.Embedding(dict_cat_class['prior_question_had_explanation'].max_len, 
                                                                     8, input_length=1)(input_prior_question_had_explanation)
embedding_prior_question_had_explanation_deep = tf.keras.layers.Lambda(squeeze)(embedding_prior_question_had_explanation_deep)

embedding_bundle_deep = tf.keras.layers.Embedding(dict_cat_class['bundle_id'].max_len,
                                             8, input_length=1)(input_bundle)
embedding_bundle_deep = tf.keras.layers.Lambda(squeeze)(embedding_bundle_deep)

embedding_part_deep = tf.keras.layers.Embedding(dict_cat_class['part'].max_len,
                                           8, input_length=1)(input_part)
embedding_part_deep = tf.keras.layers.Lambda(squeeze)(embedding_part_deep)

# 合并类别特征对应的embeeding特征和浮点特征
embedding_all = [input_timestamp,input_prior_question_elapsed_time,\
                embedding_user_deep, embedding_content_deep, embedding_task_container_deep,\
                embedding_prior_question_had_explanation_deep, embedding_bundle_deep, embedding_part_deep]

deep_all = embedding_all 

# wide layer
deep_layer = tf.keras.layers.Lambda(concat)(deep_all) 

# fc 
for unit_i in [16,256,16]:
    deep_layer = tf.keras.layers.Dense(1, activation=tf.nn.relu)(deep_layer)

# embedding_all 50 * 16 * 256 * 16


# 合并 Wide部分 和 Deep部分 接fc
deep_wide_layer = tf.keras.layers.Lambda(concat)([deep_layer,wide_layer]) 
    
# logit = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(deep_wide_layer)
logit = tf.expand_dims(tf.nn.sigmoid(tf.reduce_sum(deep_wide_layer,axis=1)),axis=1)

# tf.nn.sigmoid(tf.reduce_sum(deep_wide_layer,axis=1))

# 模型输入和输出

model = tf.keras.models.Model(inputs=inputs, outputs=logit)

In [16]:
# 编译模型
model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['binary_crossentropy'])

# 设置学习率自动0.1减少函数
plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                            verbose=1,
                            mode='min',
                            factor=0.1,
                            patience=6)

# 设置早停函数
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                               verbose=1,
                               mode='min',
                               patience=10)

# 设置保存点
checkpoint = tf.keras.callbacks.ModelCheckpoint(f'fold.h5',
                             monitor='val_loss',
                             verbose=1,
                             mode='min',
                             save_best_only=True)

# 查看模型结构
model.summary()


Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 1)]          0                                            
_______________________________________________________________________________________

In [17]:
# 验证数据获取、剩下的数据为训练数据
valid = pd.DataFrame()
for i in range(6):
    
    # 获取验证标签数据
    last_records = train.drop_duplicates('user_id', keep='last')
    
    # 获取验证标签以前的数据
    map__last_records__user_row = dict(zip(last_records['user_id'],last_records['row_id']))
    train['filter_row'] = train['user_id'].map(map__last_records__user_row)
    train = train[train['row_id']<train['filter_row']]

    # 特征加入验证集
    valid = valid.append(last_records)
    print(len(valid))

# 数据特征切分以及删除train、valid，减少内存

# 训练模型特征
features_columns = ['timestamp','prior_question_elapsed_time',\
                    'user_id','content_id',\
                    'task_container_id','prior_question_had_explanation',\
                    'bundle_id','part']

# 训练和测试集数据切分
X_valid, y_valid = [valid[columns].values for columns in features_columns], valid['answered_correctly'].values
# del valid
X_train, y_train = [train[columns].values for columns in features_columns], train['answered_correctly'].values
# del train
# 训练模型

3819
7626
11432
15236
19034
22819


In [18]:
model.fit(X_train, y_train,
          epochs=10,
          batch_size=512 * 500 * 2,
          verbose=1,
          shuffle=True,
          validation_data=(X_valid, y_valid),
          callbacks=[plateau, early_stopping, checkpoint])

# 测试集验证结果
y_valid_proba = model.predict(X_valid, verbose=0, batch_size=512)
auc = roc_auc_score(y_valid, y_valid_proba)
print(auc)

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.6715 - binary_crossentropy: 0.6715
Epoch 00001: val_loss improved from inf to 0.67972, saving model to fold.h5
1/1 [==============================] - 0s 366ms/step - loss: 0.6715 - binary_crossentropy: 0.6715 - val_loss: 0.6797 - val_binary_crossentropy: 0.6797
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.6706 - binary_crossentropy: 0.6706
Epoch 00002: val_loss improved from 0.67972 to 0.67911, saving model to fold.h5
1/1 [==============================] - 0s 100ms/step - loss: 0.6706 - binary_crossentropy: 0.6706 - val_loss: 0.6791 - val_binary_crossentropy: 0.6791
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.6696 - binary_crossentropy: 0.6696
Epoch 00003: val_loss improved from 0.67911 to 0.67848, saving model to fold.h5
1/1 [==============================] - 0s 104ms/step - loss: 0.6696 - binary_crossentropy: 0.6696 - val_loss: 0.6785 - val_binary_crossentropy: 0.6785
Ep

In [19]:
# 继续训练和验证
model.fit(X_train, y_train,
          epochs=1,
          batch_size=512 * 500 * 2,
          verbose=1,
          shuffle=True,
          validation_data=(X_valid, y_valid),
          callbacks=[plateau, early_stopping, checkpoint])

y_valid_proba = model.predict(X_valid, verbose=0, batch_size=512)
auc = roc_auc_score(y_valid, y_valid_proba)
print(auc)
#%% md
# 测试集环境
#%%
# iter_test = env.iter_test()
#%%
# for (test_df, sample_prediction_df) in iter_test:

#     # 处理特征
#     test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].map({'True':1,'False':0}).fillna(-1).astype(np.int8)

#     # 合并样本
#     test_df = pd.merge(
#         left=test_df,
#         right=questions,
#         how='left',
#         left_on='content_id',
#         right_on='question_id'
#         )

#     test_df = test_df.fillna(0)


#     for columns in ['user_id','content_id',\
#                     'task_container_id','prior_question_had_explanation',\
#                     'bundle_id','part']:

#         test_df[columns] = dict_cat_class[columns].transform(test_df[columns])
#         print(columns)


#     for columns in ['timestamp','prior_question_elapsed_time']:
      
#         test_df[columns] = dict_float_class[columns].transform(test_df[columns])
#         print(columns)

#     X_test = [test_df[columns].values for columns in features_columns]

#     test_df['answered_correctly'] =  model.predict(X_test, verbose=0, batch_size=512)
#     env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])



1/1 [==============================] - ETA: 0s - loss: 0.6616 - binary_crossentropy: 0.6616
Epoch 00001: val_loss improved from 0.67410 to 0.67348, saving model to fold.h5
1/1 [==============================] - 0s 155ms/step - loss: 0.6616 - binary_crossentropy: 0.6616 - val_loss: 0.6735 - val_binary_crossentropy: 0.6735
0.6319446535373028
